In [1]:
%pip install Bio

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Importing libraries
import os
import sys
import json
from Bio import SeqIO
import itertools


In [3]:
# Import Files
fasta_path = "sample.fa"
record_list = list(SeqIO.parse(fasta_path, "fasta"))


In [4]:
# Metadata
output_file = open('nucleotide_counts.tsv', 'w')
output_file.write('Gene\tA\tC\tG\tT\tLength\tCG%\n')
with open(fasta_path) as handle:
    for seq_record in SeqIO.parse(handle, "fasta"):
        # count nucleotides in this record...
        gene_name = seq_record.name
        A_count = seq_record.seq.count('A')
        C_count = seq_record.seq.count('C')
        G_count = seq_record.seq.count('G')
        T_count = seq_record.seq.count('T')
        length = len(seq_record.seq)
        cg_percentage = float(C_count + G_count) / length
        output_line = '%s\t%i\t%i\t%i\t%i\t%i\t%f\n' % (
            gene_name, A_count, C_count, G_count, T_count, length, cg_percentage)
        output_file.write(output_line)


In [5]:
seq_list = []
windowWidth = 4

for i in record_list:
    seq_list.append(str(i.seq))
print(len(seq_list))


4


In [6]:
def VariableGenerator(length, string):
    nucleotides = ["A", "T", "G", "C"]
    iterproduct = itertools.product(nucleotides, repeat=length)
    list = [''.join(iterproduct) for iterproduct in iterproduct]
    var_list = []
    for item in list:
        variable = str(item+string)
        var_list.append(variable)

    return var_list


In [7]:
# Sliding Window function
def slidingBlock(seq_list, n):
    if windowWidth > n:
        TheMegaList = []
        Block_size = int((windowWidth - n + 1) * len(seq_list))
        Block_score_list_variables = VariableGenerator(n, "_Block_score_list")
        Nucleic_acid_list = VariableGenerator(n, "")
        for var in Block_score_list_variables:
            globals()[var] = []

        # Looping through the list of sequences
        for seq in seq_list:
            length = len(seq)
            Block_score_variables = VariableGenerator(n, "_Block_score")
            for var in Block_score_variables:
                globals()[var] = []

            # Loop through entire sequence
            for i in range(length - windowWidth + 1):
                Count_variables = VariableGenerator(n, "Count")
                for var in Count_variables:
                    globals()[var] = 0

                # In each iteration, loop though window
                for j in range(i, windowWidth + i):
                    for na in Nucleic_acid_list:
                        if seq[j:j+n] == na:
                            globals()[na+"Count"] += 1

                # Update counts of each nucleotide in respective dictionary
                for z in Nucleic_acid_list:
                    a = globals()[str(z+"_Block_score")]
                    b = globals()[str(z+"Count")]
                    a.append(b/Block_size)

            for y in Nucleic_acid_list:
                c = globals()[y+"_Block_score_list"]
                d = globals()[y+"_Block_score"]
                c.append(d)

        for y in Nucleic_acid_list:
            e = globals()[y+"_Block_score_list"]
            TheMegaList.append(e)
        return TheMegaList, Nucleic_acid_list
    else:
        return 0


In [8]:
TheMegaList, Nucleic_acid_list = slidingBlock(seq_list, int(2))


In [9]:
seq_dict = {}
for i in range(len(Nucleic_acid_list)):
    a = Nucleic_acid_list[i]
    seq_dict[a] = TheMegaList[i]


In [10]:
# Exporting the output as JSON
with open("output.json", "w") as outfile:
    json.dump(seq_dict, outfile)
